# モジュールをインポート

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# モデルを定義

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
results_dir = "./results/keras_2019_12_23_12_11"

In [ ]:
model = load_model(results_dir + "/model/model.h5")

# モデルの確認

In [ ]:
model.summary()

In [ ]:
# opt = Adam(lr=learning_rate)
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## 学習済みモデルの重みを読み込み

In [ ]:
# checkpoint_path = "./results/keras_2019_11_21/keras_model/cp.ckpt"
# model.load_weights(checkpoint_path)

# Load data stream

In [ ]:
from pylsl import StreamInlet, resolve_byprop
from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK

LSL_EEG_CHUNK = 10
def record_stream():
    """
    :return: a generator to fetch EEG data from existed stream.
    """
    streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
    if len(streams) == 0:
        raise IOError("Can't find EEG stream.")

    inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

    info = inlet.info()
    description = info.desc()
    Nchan = info.channel_count()

    ch = description.child('channels').first_child()
    ch_names = [ch.child_value("label")]
    for i in range(1, Nchan):
        ch = ch.next_sibling()
        ch_names.append(ch.child_value('label'))
#     print(ch_names)
    while True:
        yield inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)


In [ ]:
print(next(record_stream()))

In [ ]:
print("['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']")
print(np.array(next(record_stream())[0])[:, 0:4])

# Preprocessing

In [ ]:
def data_1Dto2D(data, Y=10, X=11):
    data_2D = np.zeros([Y, X])
    data_2D[1, 3] = data[1]
    data_2D[1, 7] = data[2]
    data_2D[5, 1] = data[0]
    data_2D[5, 9] = data[3]
# 	data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,        0,        0,        0, 	   0,  	     0, 	   0, 	 	 0) 
# 	data_2D[1] = (	  	 0, 	   0,  	   	 0, data[1],        0,        0,        0, data[2], 	   	 0,   	   0, 	 	 0) 
# 	data_2D[2] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 	 	 0) 
# 	data_2D[3] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[4] = (       0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0) 
# 	data_2D[5] = (	  	 0, data[0],        0,        0,        0,        0,        0,        0,        0, data[3], 		 0) 
# 	data_2D[6] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[7] = (	  	 0, 	   0, 	 	 0,        0,        0,        0,        0,        0, 	   	 0, 	   0, 		 0) 
# 	data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0,        0,        0,        0, 	   0, 	   	 0, 	   0, 		 0) 
# 	data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0) 
    return data_2D

In [ ]:
buffer_2d = np.zeros((10, 10, 11))

In [ ]:
X_test = buffer_2d.reshape((1, 10, 10, 11, 1))

In [ ]:
true_labels = np.array(['eye_close', 'image_open&close_both_feet',
       'image_open&close_both_fists', 'image_open&close_left_fist',
       'image_open&close_right_fist'])

In [ ]:
def feature_normalize(data):
	mean = data[data.nonzero()].mean()
	sigma = data[data.nonzero()].std()
	data_normalized = data
	data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
	return data_normalized

In [ ]:
for stream in record_stream():
    buffer_3d = []
    buffer_2d = np.zeros((10, 11))
    num = 0
    for data in stream[0]:
        data = np.array(data)
        data = feature_normalize(data)
        buffer_2d[1, 3] = data[1]
        buffer_2d[1, 7] = data[2]
        buffer_2d[5, 1] = data[0]
        buffer_2d[5, 9] = data[3]
        buffer_3d.append(buffer_2d)
    buffer_3d = np.array(buffer_3d)
    X_test = buffer_3d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
#     print(y_pred.round(3), end="\r", flush=True)
    y_pred = (y_pred > 0.8)
    print(true_labels[y_pred], end="\r", flush=True)
    
#     y_pred_bool = np.argmax(y_pred, axis=1)
#     print(true_labels[int(y_pred_bool)], end="\r", flush=True)
#     print(true_labels[int(y_pred_bool)])

In [ ]:
for stream in record_stream():
    buffer_2d = np.zeros((10, 10, 11))
#     print(stream[0])
    for i in next(record_stream())[0]:
        num = 0
        buffer_2d[num, 1, 3] = i[1]
        buffer_2d[num, 1, 7] = i[2]
        buffer_2d[num, 5, 1] = i[0]
        buffer_2d[num, 5, 9] = i[3]
    X_test = buffer_2d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(true_labels[int(y_pred_bool)])

# テストデータを分類

In [ ]:
y_pred = model.predict(X_test, batch_size=1, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

In [ ]:
print("サンプリング率:", 160)
print("1秒に処理できる信号数：", 990*10-6)

In [ ]:
y_pred_bool = one_hot_encoder(y_pred_bool)
y_pred_bool

# 精度と正解率の曲線

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.preprocessing import label_binarize

precision = dict()
recall = dict()
for i in range(5):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_pred_bool[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

# 分類結果

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_bool))

In [ ]:
print("検証正解率：", history["val_acc"][-1])

In [ ]:
from sklearn.metrics import accuracy_score
print("テスト正解率：", accuracy_score(y_test, y_pred_bool))